<a href="https://colab.research.google.com/github/JSJeong-me/AI-Innovation-2024/blob/main/NLP/4-7-LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences


MDB 데이터셋을 사용하여 RNN을 훈련합니다. 이 데이터는 영화 리뷰에 대한 긍정/부정 레이블이 포함되어 있습니다.

In [2]:
# Hyperparameters
max_features = 10000  # 사용할 단어의 최대 개수
max_len = 300  # 패딩 후 시퀀스의 길이
embedding_dim = 64

# Load IMDB dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# Pad sequences to the same length
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


이 부분에서는 LSTM 레이어에 드롭아웃, 배치 정규화, 그래디언트 클리핑 등의 기법을 적용하여 모델을 구축합니다.

In [3]:
def create_model():
    model = Sequential()

    # LSTM Layer with Dropout
    model.add(tf.keras.layers.Embedding(max_features, embedding_dim, input_length=max_len))
    model.add(LSTM(128, return_sequences=True))  # 첫 번째 LSTM 층
    model.add(Dropout(0.5))  # 드롭아웃 적용

    # Batch Normalization
    model.add(BatchNormalization())

    # Second LSTM layer with Dropout
    model.add(LSTM(64))
    model.add(Dropout(0.5))

    # Output Layer
    model.add(Dense(1, activation='sigmoid'))

    return model

# Create model
model = create_model()

# Compile model with gradient clipping
optimizer = Adam(learning_rate=0.001, clipnorm=1.0)  # Gradient clipping 적용
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

모델을 학습시키며, 검증 데이터로 x_test와 y_test를 사용합니다.

In [4]:
# Train model
history = model.fit(
    x_train,
    y_train,
    epochs=5,
    batch_size=64,
    validation_data=(x_test, y_test),
    verbose=2
)


Epoch 1/5
391/391 - 30s - 78ms/step - accuracy: 0.7537 - loss: 0.4852 - val_accuracy: 0.8232 - val_loss: 0.5046
Epoch 2/5
391/391 - 29s - 74ms/step - accuracy: 0.8923 - loss: 0.2822 - val_accuracy: 0.8697 - val_loss: 0.3120
Epoch 3/5
391/391 - 22s - 56ms/step - accuracy: 0.9194 - loss: 0.2156 - val_accuracy: 0.8510 - val_loss: 0.3762
Epoch 4/5
391/391 - 19s - 49ms/step - accuracy: 0.9393 - loss: 0.1755 - val_accuracy: 0.8617 - val_loss: 0.3861
Epoch 5/5
391/391 - 14s - 37ms/step - accuracy: 0.9559 - loss: 0.1349 - val_accuracy: 0.8354 - val_loss: 0.5004


훈련된 모델을 평가합니다.

In [5]:
# Evaluate model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'Test Accuracy: {test_acc:.4f}')


782/782 - 7s - 10ms/step - accuracy: 0.8354 - loss: 0.5004
Test Accuracy: 0.8354


In [9]:
# !pip install keras
# Import the required module
from tensorflow.keras.preprocessing.text import Tokenizer

# 예측 예제
sample_text = ["This movie was absolutely fantastic! The performances were breathtaking and the story was captivating."]
# 임의의 텍스트를 인덱스로 변환해야 함: 여기서는 임베딩 된 단어로 인덱싱 된다고 가정

# 텍스트를 토큰화하고 인덱스로 변환 (여기에 적절한 토큰화 및 인덱싱 로직 추가)
# 예:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sample_text)
sample_indices = tokenizer.texts_to_sequences(sample_text)

# 예제 데이터를 위한 패딩 처리
# Make sure you have defined 'pad_sequences' and 'max_len'
sample_input = pad_sequences(sample_indices, maxlen=max_len)

# 예측
prediction = model.predict(sample_input)
print(f'Predicted sentiment: {"Positive" if prediction > 0.5 else "Negative"}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted sentiment: Negative
